In [6]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multiple Layer LSTM

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.4.1


In [8]:
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.4.1


In [9]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


In [10]:
tokenizer = info.features['text'].encoder

In [11]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 130s 307ms/step - loss: 0.6252 - accuracy: 0.6371 - val_loss: 0.6228 - val_accuracy: 0.6991
Epoch 2/10
391/391 [==============================] - 119s 304ms/step - loss: 0.4910 - accuracy: 0.7827 - val_loss: 0.4920 - val_accuracy: 0.7674
Epoch 3/10
391/391 [==============================] - 119s 303ms/step - loss: 0.3624 - accuracy: 0.8478 - val_loss: 0.4650 - val_accuracy: 0.7761
Epoch 4/10
391/391 [==============================] - 119s 304ms/step - loss: 0.3201 - accuracy: 0.8727 - val_loss: 0.4066 - val_accuracy: 0.8392
Epoch 5/10
391/391 [==============================] - 120s 305ms/step - loss: 0.2457 - accuracy: 0.9027 - val_loss: 0.5521 - val_accuracy: 0.7402
Epoch 6/10
391/391 [==============================] - 123s 314ms/step - loss: 0.2910 - accuracy: 0.8856 - val_loss: 0.4273 - val_accuracy: 0.8287
Epoch 7/10
391/391 [==============================] - 123s 314ms/step - loss: 0.1658 - accuracy: 0.9401 - val_loss: 0.4219 -

In [16]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [17]:
plot_graphs(history, 'accuracy')

In [18]:
plot_graphs(history, 'loss')